In [23]:
from tqdm import tqdm_notebook as tqdm 
import numpy as np
from collections import deque, defaultdict, OrderedDict


In [147]:
index = 0
frame = 4
cache = deque(maxlen=int(frame/2))
FIFOcache = deque(maxlen=frame)
LRUcache = deque(maxlen=frame)
LFUcache = deque(maxlen=frame)
frequency = defaultdict(int)
weight = defaultdict(int)
hit, miss = 0, 0 

In [81]:
LRUCache

deque([1, 2, 3, 4])

In [146]:
blocktrace = np.random.randint(15, size=(50000))
blocktrace

array([ 4,  3, 14, ...,  9,  4, 14])

In [139]:
weight['LFU'] = 10

In [148]:
for i in blocktrace:
    checkVirtualCache(i)
    if i in cache:
        hit += 1
        cache.remove(i)
        cache.append(i)
    
    else:
        miss += 1
        
        policy = max(weight, key=weight.get)
        #print(policy)
        if policy == 'FIFO':
            #print('FIFO')
            cache.append(i)
        
        elif policy == 'LFU':
            #print('LFU')
            tempFrequency = {k:j for k,j in frequency.items() if k in cache}
            try:
                cache.remove(min(tempFrequency, key=tempFrequency.get))
            except:
                pass
            cache.append(i)
        
        else:
            #print('LRU')
            cache.append(i)

In [134]:
def checkVirtualCache(newBlock):
    global frequency, weight, LRUcache, LFUcache, FIFOcache
    frequency[newBlock] += 1
    
    if newBlock in LRUcache:
        weight["LRU"] += 1
        LRUcache.remove(newBlock)
        LRUcache.append(newBlock)
    else:
        weight["LRU"] -= 1
        LRUcache.append(newBlock)
    
    if newBlock in LFUcache:
        weight["LFU"] += 1
    else:
        weight["LFU"] -= 1
        tempFrequency = {i:j for i,j in frequency.items() if i in LFUcache}
        try:
            LFUcache.remove(min(tempFrequency, key=tempFrequency.get))
        except:
            pass
        LFUcache.append(newBlock)
        
    if newBlock in FIFOcache:
        weight["FIFO"] += 1
    else:
        weight["FIFO"] -= 1
        FIFOcache.append(newBlock)

In [102]:
max(d, key=d.get)

'a'

In [97]:
d = defaultdict(int)
d['b'] = 1
d['a'] = 2
d['c'] = 2

In [101]:
d['a'] += 1

In [149]:
hit / (hit + miss)

0.13124

In [142]:
def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            return cached_block            
    for cached_block in C:
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    return maxAccessBlock

def belady_opt(blocktrace, frame):
    OPT = defaultdict(deque)

    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block].append(i)    

    #print ("created OPT dictionary")    

    hit, miss = 0, 0

    C = set()
    seq_number = 0
    for block in tqdm(blocktrace, desc="OPT"):

        if block in C:
            #OPT[block] = OPT[block][1:]
            OPT[block].popleft()
            hit+=1
            #print('hit' + str(block))
            #print(OPT)
        else:
            #print('miss' + str(block))
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                C.remove(fblock)
            C.add(block)
            #OPT[block] = OPT[block][1:]
            #print(OPT)
            OPT[block].popleft()

    #print ("hit count" + str(hit_count))
    #print ("miss count" + str(miss_count))
    hitrate = hit / (hit + miss)
    #print(hitrate)
    return hitrate

In [150]:
belady_opt(blocktrace, 2)

0.27828

In [144]:
def LRU(blocktrace, frame):
    cache = set()
    recent_block = np.array([], dtype=int)
    
    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in tqdm(blocktrace, desc="LRU", leave=False):
    
        if block in cache:
            recent_block = np.delete(recent_block, np.argwhere(recent_block==block))
            recent_block = np.append(recent_block, block)
            hit += 1
            
        
        elif len(cache) < frame:
            recent_block = np.append(recent_block, block)
            cache.add(block)
            miss += 1

        else:
            cache.remove(recent_block[0])
            recent_block = recent_block[1:]
            recent_block = np.append(recent_block, block)
            cache.add(block)
            miss += 1
        #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    #print("\n\nhitrate: ", hitrate)
    return hitrate

In [151]:
LRU(blocktrace, 2)

0.13124

In [152]:
def LRU(blocktrace, frame):
    cache = set()
    recent_block = np.array([], dtype=int)
    
    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in tqdm(blocktrace, desc="LRU", leave=False):
    
        if block in cache:
            recent_block = np.delete(recent_block, np.argwhere(recent_block==block))
            recent_block = np.append(recent_block, block)
            hit += 1
            
        
        elif len(cache) < frame:
            recent_block = np.append(recent_block, block)
            cache.add(block)
            miss += 1

        else:
            cache.remove(recent_block[0])
            recent_block = recent_block[1:]
            recent_block = np.append(recent_block, block)
            cache.add(block)
            miss += 1
        #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    #print("\n\nhitrate: ", hitrate)
    return hitrate

In [153]:
LRU(blocktrace, 5)

0.33042